In [1]:
import requests
from bs4 import BeautifulSoup

In [36]:
def get_regions(page = "https://pronounce.voanews.com/browse-region.php"):
    region_page = requests.get(page)
    soup = BeautifulSoup(region_page.content, "html.parser")
    regions = []
    for pane in soup.find_all("div", {"class": "tab-pane"}):
        for link in pane.find_all("a"):
            regions.append("https://pronounce.voanews.com/" + link.get("href"))
    return regions

In [34]:
import re

def get_region_pronunciations(region_url):
    BASE = "https://pronounce.voanews.com"
    region_page = requests.get(region_url)
    soup = BeautifulSoup(region_page.content, "html.parser")
    title = soup.find("h2").text.replace("Browse Region - ", "").strip()
    pronunciations = []
    # body > div.container > div.content > div > table
    for item in soup.find_all("table", {"class": "table table-condensed table-hover"}):
        for row in item.find_all("tr"):
            cells = row.find_all("td")
            # there are three cells; the first contains <a class="btn icon playLink"> with the audio
            # the second contains the word, and the third contains the pronunciation; the HTML of the third cell is significant
            if len(cells) == 3:
                audio_link = cells[0].find("a", {"class": "btn icon playLink"})
                word = cells[1].text.strip()
                pronunciation_html = cells[2]
                # the pronunciation is contained in the HTML of the third cell, but we need to extract the HTML
                pronunciation_html = pronunciation_html.decode_contents().strip()
                pronunciation = cells[2].text.strip().replace('\xa0', ' ')
                pronunciation = re.sub(r'\s+', ' ', pronunciation)
                pronunciations.append({
                    "word": word,
                    "pronunciation_html": pronunciation_html,
                    "pronunciation": pronunciation,
                    "audio_link": BASE + audio_link.get("href") if audio_link else None,
                    "region": title
                })
            else:
                print(row)
    return pronunciations

In [35]:
get_region_pronunciations(SAMPLE)

[{'word': 'ANKVAB, ALEXANDER',
  'pronunciation_html': 'ah-lehk-<b>SAHN-</b>dehr\xa0\xa0\xa0<b>AHNK-</b>vahb',
  'pronunciation': 'ah-lehk-SAHN-dehr AHNK-vahb',
  'audio_link': 'https://pronounce.voanews.com/sounds/5269.mp3',
  'region': 'Abkhazia'},
 {'word': 'ARDZINGBA, VLADISLAV',
  'pronunciation_html': '<b>VLAHD</b>-ih-slahv<b>\xa0\xa0\xa0ART</b>-zing-bah',
  'pronunciation': 'VLAHD-ih-slahv ART-zing-bah',
  'audio_link': 'https://pronounce.voanews.com/sounds/146.mp3',
  'region': 'Abkhazia'},
 {'word': 'BAGAPSH, SERGEI',
  'pronunciation_html': 'sir-<b>GAY\xa0\xa0\xa0</b>bah-<b>GAHPSH</b>',
  'pronunciation': 'sir-GAY bah-GAHPSH',
  'audio_link': 'https://pronounce.voanews.com/sounds/5037.mp3',
  'region': 'Abkhazia'},
 {'word': 'DZHERGENIYA, ANRI',
  'pronunciation_html': '<b>AHN-</b>ree\xa0\xa0\xa0\xa0jehr-<b>GHEH-</b>nee-ah',
  'pronunciation': 'AHN-ree jehr-GHEH-nee-ah',
  'audio_link': 'https://pronounce.voanews.com/sounds/2565.mp3',
  'region': 'Abkhazia'},
 {'word': 'KHADZ